# FDMNES "01_calc_radii"

This script is part of a collection of Jupyer notebooks for fdmnes.  

**For a given structure in XYZ format, calculate a set of inter-atomic distances to be tried during radius convergence check.**

Usage: 

 -  Use VESTA (or another software) to create XYZ structure from CIF file. Include a few unit cells in each direction.
 -  Adapt this notebook. Set:
    -  filepath of XYZ file (1)
    -  absorbing atom (2)
    -  min_distance and max_distance (inter-atomic distances) (3)
    -  number N of desired intervals (4) - Will choose the N largest space-dividing intervals

## Functions

In [2]:
import re
import numpy as np
import json
import os
import matplotlib.pyplot as plt
%matplotlib widget

def read_atoms(filename):
    atoms = []
    
    pattern = r'(\w+)\s+([+-]?\d+\.?\d*)\s+([+-]?\d+\.?\d*)\s+([+-]?\d+\.?\d*)'
#     pattern = r'\|\s*\d+>\s+(\w{1,2})\s+([+-]?\d+\.\d+)\s+([+-]?\d+\.\d+)\s+([+-]?\d+\.\d+)'
    with open(filename, 'r') as file:
        for line in file.readlines():
            m = re.search(pattern, line)
            
            if m:
                name, x, y, z = m.groups()
                atoms.append([name, (float(x), float(y), float(z))])     
                
    return dict(zip(range(len(atoms)), atoms))
                

def get_distances(atom, atoms):
    """ Compute the distance from *atom* to all atoms in dict *atoms* """
    
    distances = []
    for idx in range(len(atoms)):
        a = atoms[idx]
        distances.append(np.sqrt(np.sum([(v-k)**2 for v, k in zip(atom[1], a[1])])))
        
    return distances
        
    
def find_neighbors(atom, atoms, max_distance, exclude_elements = []):
    """ Find neighbors in the vicinity of *atom* and return them as dict. """
    
    distances = get_distances(atom, atoms)
    
    # Filter by distance
    neighbor_idx = [i for i in range(len(atoms)) if distances[i] <= max_distance]
    
    # Filter by elements
    neighbor_idx = [idx for idx in neighbor_idx if atoms[idx][0] not in exclude_elements]
    
    
    
    return dict([(idx, atoms[idx]) for idx in neighbor_idx])


def get_fragment(atom, atoms, max_distance, exclude_elements = [], parent_fragment = {}):
    """ Recursively add atoms to a fragment dict based on *max_distance* and  
        *exclude_elements* until no new atoms are found. Start from *atom*.
    """
    
    neighbors = find_neighbors(atom, atoms, max_distance, exclude_elements)
    
    fragment = {}
    
    for idx, neighbor in neighbors.items():
        
        if idx in {**parent_fragment, **fragment}.keys():
            continue
        else:
            fragment[idx] = neighbor
        
        if neighbor[0] != 'H':
            # We came here somehow. H should only be allowed to have one bonding partner.
            sub_neighbors = get_fragment(neighbor, atoms, max_distance, exclude_elements, {**fragment, **parent_fragment})
            fragment = {**fragment, **sub_neighbors}
    
    return fragment
    

## Calculate radii

Set filepath **(1)**

In [4]:
# Read all atoms from XYZ file
base = r'/home/esrf/USERNAME/fdmnes/u-1/kuo3'
atoms = read_atoms(os.path.join(base, '71241.xyz'))

Set absorbing atom **(2)**

In [5]:
# Compute all distances from all atoms of absorbing species
absorber = 'O'  
distances = []
for element, coords in atoms.values():
    if element != absorber:
        continue
        
    distances.extend( get_distances((element, coords), atoms) )
        
# Round to decimals
distances = np.round(distances, 1)

# Compute unique distances
distances = np.unique(distances)

Set min and max distance **(3)**

In [6]:
# Return all distances between min_distance and max_distance
min_distance = 0.25
max_distance = 10
distances = [d for d in distances if d <= max_distance and d >= min_distance]

Set number of desired intervals **(4)**

In [7]:
intervals = 15
bins = np.diff(distances)/2 + distances[:-1]
gaps = np.argsort(np.diff(distances))
bins = bins[gaps][::-1][:intervals]
bins = sorted(bins.tolist())
bins.append(max_distance)

Plot the results and print the intervals

In [8]:
plt.figure()
plt.bar(distances, np.ones(len(distances)), width = 0.01)
plt.bar(bins, np.ones(len(bins))*1.2, width = 0.05, color = 'r')
print(bins)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[2.55, 3.65, 4.55, 5.05, 5.699999999999999, 6.25, 6.6, 7.1, 7.550000000000001, 7.85, 8.3, 8.75, 9.0, 9.35, 9.7, 10]
